### In this notebook we hava created a scraping tool for tripadvisor. This notebooks is an addition to the review scarping tool to gain additional information about restaurants and reviewers.

In [15]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import csv 
from selenium import webdriver
import time
import string
import pickle
import os

In [16]:
# GEt all the reviwer info, location, join date, review count, upvotes, followers and following.
def reviewerInfo(url):
    username = url
    full_url = f"https://www.tripadvisor.com/Profile/{url}"
    driver.get(full_url)
    time.sleep(1)

    # Get Intro info, location and join date
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        location = soup.find('span', class_ = "_2VknwlEe _3J15flPT default").text
    except:
        location = None

    try:
        joined = soup.find('span', class_ = "_1CdMKu4t").text
    except:
        joined = None


    all_links = soup.find_all('div', class_ = '_1aVEDY08')
    # link = driver.find_elements_by_xpath("//div[@class='nkw-3XeH']/div[1]/span[2]/a")

    # # Get the contributions info
    nrContributions = int(str(all_links[0].text).split()[1])
    if nrContributions > 0:
        link = driver.find_elements_by_xpath("//div[@class='nkw-3XeH']/div[1]/span[2]/a")
        driver.execute_script("arguments[0].click();", link[0])
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        nrReviews = int(str(soup.find('span', class_ = 'ui_icon pencil-paper _1LSVmZLi').parent.text).split()[0])
        try:
            nrUpvotes = int(str(soup.find('span', class_ ='ui_icon thumbs-up _1LSVmZLi _3zmXi7gU').parent.text).split()[0])
        except:
            nrUpvotes = 0
        close = driver.find_elements_by_xpath("//div[@class='_2EFRp_bb _9Wi4Mpeb']")
        driver.execute_script("arguments[0].click();", close[0])
    else:
        nrReviews = 0
        nrUpvotes = 0

    # Get Followers
    nrFollowers = int(str(all_links[1].text).split()[1])
    if nrFollowers > 0:
        link = driver.find_elements_by_xpath("//div[@class='nkw-3XeH']/div[2]/span[2]/a")
        driver.execute_script("arguments[0].click();", link[0])
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        followers = [word.text for word in soup.find('div', class_='_1caczhWN').find_all('span', class_='gf69u3Nd')]
        close = driver.find_elements_by_xpath("//div[@class='_2EFRp_bb _9Wi4Mpeb']")
        driver.execute_script("arguments[0].click();", close[0])
    else:
        followers = []

    # Get all following
    nrFollowing = int(str(all_links[2].text).split()[1])
    if nrFollowing > 0:
        link = driver.find_elements_by_xpath("//div[@class='nkw-3XeH']/div[3]/span[2]/a")
        driver.execute_script("arguments[0].click();", link[0])
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        following = [word.text for word in soup.find('div', class_='_1caczhWN').find_all('span', class_='gf69u3Nd')]
        close = driver.find_elements_by_xpath("//div[@class='_2EFRp_bb _9Wi4Mpeb']")
        driver.execute_script("arguments[0].click();", close[0])
    else:
        following = []

    with open(pathtoReviewers, mode='a', encoding="utf-8") as reviewer_data:
        data_writer = csv.writer(reviewer_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        data_writer.writerow([username, location, joined, nrContributions,nrReviews, nrUpvotes, nrFollowers, followers, nrFollowing,following])

In [5]:
pathtoReviewers = "reviewerInfo_kristin.csv"
with open(pathtoReviewers, mode='a', encoding="utf-8") as reviewer_data:
    data_writer = csv.writer(reviewer_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    data_writer.writerow(['username', 'location', 'joined', 'nrContributions','nrReviews', 'nrUpvotes', 'nrFollowers','followers', 'nrFollowing','following'])


In [ ]:
driver_path = f'{os.getcwd()}/chromedriver'
driver = webdriver.Chrome(driver_path)

with open("all_reviewers.txt", "rb") as f:   # Unpickling
    reviewers = pickle.load(f)

reviewer_slice = reviewers[0:25000]
bad_url = []
i = 0
for reviewer in reviewer_slice:
    try:
        reviewerInfo(reviewer)
        if i % 10 == 0:
            print(i)
        i+=1
    except:
        bad_url.append(reviewer)
        if i % 10 == 0:
            print(i)
        i+=1